In [42]:
import ScraperFC as sfc  # LEAVE THIS
from datetime import datetime
import pandas as pd
from tqdm import tqdm
import  time
import logging

In [43]:
logging.basicConfig(level=logging.DEBUG)

In [44]:
ss = sfc.Sofascore()

In [45]:
LEAGUES = {
    "EPL":17,
    "La Liga":8,
    "Liga Portugal Betclic":238,
    "Ligue 1":34,
    "Bundesliga":35,
    'Serie A':23,
    "Champions League":7,
    "brasileirao-serie-a":325,
    "MLS":242,
    "championship":18,
    'Europa League':679,
    'Copa Del Rey' :329,
    'EFL Cup':21,
    'Premiership':36,   
}

In [46]:
# Define the seasons and leagues

seasons = ['23/24', '24/25']
calendar_year_seasons = [str(year) for year in range(2023, 2025)]  # Use calendar years for MLS, Brasileirao

leagues = ['EPL', 'La Liga', 'liga-portugal-betclic', 'Ligue 1', 'Bundesliga', 'Serie A', 'Champions League', 'MLS', 'brasileirao-serie-a', 'championship', 'Europa League']

all_events = []


# Loop through each league
for league in leagues:
    try:
        # Check if the league uses calendar years
        if league in ['MLS', 'brasileirao-serie-a']:
            # Fetch match data using calendar year format
            for calendar_season in calendar_year_seasons:
                events = ss.get_match_dicts(calendar_season, league)
                all_events.extend(events)
                print(f"Successfully fetched data for {league} - {calendar_season}")
        else:
            # Fetch match data using 'XX/XX' format for European leagues
            for season in seasons:
                events = ss.get_match_dicts(season, league)
                all_events.extend(events)
                print(f"Successfully fetched data for {league} - {season}")

    except Exception as e:
        # Handle any exceptions that may occur during the fetching process
        print(f"Error fetching data for {league}: {e}")

# Now 'all_events' contains all the match data for the specified leagues and seasons


Successfully fetched data for EPL - 23/24
Successfully fetched data for EPL - 24/25
Successfully fetched data for La Liga - 23/24
Successfully fetched data for La Liga - 24/25
Successfully fetched data for liga-portugal-betclic - 23/24
Successfully fetched data for liga-portugal-betclic - 24/25
Successfully fetched data for Ligue 1 - 23/24
Successfully fetched data for Ligue 1 - 24/25
Successfully fetched data for Bundesliga - 23/24
Successfully fetched data for Bundesliga - 24/25
Successfully fetched data for Serie A - 23/24
Successfully fetched data for Serie A - 24/25
Successfully fetched data for Champions League - 23/24
Successfully fetched data for Champions League - 24/25
Successfully fetched data for MLS - 2023
Successfully fetched data for MLS - 2024
Successfully fetched data for brasileirao-serie-a - 2023
Successfully fetched data for brasileirao-serie-a - 2024
Successfully fetched data for championship - 23/24
Successfully fetched data for championship - 24/25
Successfully f

In [61]:
matches_data = []

In [62]:
for event in tqdm(all_events, desc="Processing events", unit="event"):
        try:
                event_id = event.get("id", None)
                tournament_name = event.get('season').get('name')
                home_team = event.get('homeTeam', {}).get('name', 'Unknown')
                home_team_id = event.get('homeTeam', {}).get('id', 'Unknown')
                away_team = event.get('awayTeam', {}).get('name', 'Unknown')
                away_team_id = event.get('awayTeam', {}).get('id', 'Unknown')
                home_score = event.get('homeScore', {}).get('display', 0)
                away_score = event.get('awayScore', {}).get('display', 0)
                
                start_timestamp = event.get("startTimestamp")
                if start_timestamp:
                        match_datetime = datetime.fromtimestamp(start_timestamp)
                        match_date = match_datetime.strftime("%Y-%m-%d")
                        match_time = match_datetime.strftime("%H:%M:%S")
                else:
                        match_date = None
                        match_time = None
                        
                gd = home_score - away_score
                if gd == 0:
                        FTR = 1
                        home_outcome = 0  # Draw
                        away_outcome = 0  # Draw
                elif gd > 0:
                        FTR = 0
                        home_outcome = 1  # Home team won
                        away_outcome = -1  # Away team lost
                elif gd < 0:
                        FTR = 2
                        home_outcome = -1  # Home team lost
                        away_outcome = 1  # Away team won
                match_data = ss.scrape_team_match_stats(event_id)
                new_row = {
                        "eventId": event_id,
                        "tournament": tournament_name,
                        "homeId": home_team_id,
                        "homeTeam": home_team,
                        "awayId": away_team_id,
                        "awayTeam": away_team,
                        "homeScore": home_score,
                        "awayScore": away_score,
                        "FTR": FTR,  # Full-Time Result
                        "date": match_date,
                        "time": match_time
                }
                match_data_filtered = match_data[['key', 'home', 'away']]
                for index, row in match_data_filtered.iterrows():
                        key = row['key']
                        # Create the new columns for home_{key} and away_{key}
                        new_row[f'home{key}'] = row['home']
                        new_row[f'away{key}'] = row['away']
                matches_data.append(new_row)

        except Exception as e:
                logging.error(f"Error processing event {event.get('id', 'Unknown')}: {e}", exc_info=True)
                continue  # Skip to the next event in case of an error

        time.sleep(1)

Processing events:  13%|█▎        | 744/5946 [15:04<1:47:09,  1.24s/event]c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\ScraperFC\sofascore.py:399: UserWarning: Scraper did not get status code 200 from Sofascore. Returning empty dataframe.
  warnings.warn(
ERROR:root:Error processing event 11369392: "None of [Index(['key', 'home', 'away'], dtype='object')] are in the [columns]"
Traceback (most recent call last):
  File "C:\Users\ahmed\AppData\Local\Temp\ipykernel_17676\2219153017.py", line 48, in <module>
    match_data_filtered = match_data[['key', 'home', 'away']]
                          ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\frame.py", line 3813, in __getitem__
    indexer = self.columns._get_indexer_strict(key, "columns")[1]
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-package

In [63]:
matches_df = pd.DataFrame(matches_data)

In [64]:
# Combine 'date' and 'time' into a single datetime column
matches_df['datetime'] = pd.to_datetime(matches_df['date'] + ' ' + matches_df['time'])
# Sort the DataFrame by the new 'datetime' column
matches_df_sorted = matches_df.sort_values(by='datetime').reset_index(drop=True)
matches_df_final = matches_df_sorted.drop_duplicates(subset='eventId').reset_index(drop=True)
matches_df_final.to_csv('europa_data_final_2.csv',index=False)


{'events': [{'tournament': {'name': 'Premier League', 'slug': 'premier-league', 'category': {'name': 'England', 'slug': 'england', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'id': 1, 'country': {'alpha2': 'EN', 'alpha3': 'ENG', 'name': 'England', 'slug': 'england'}, 'flag': 'england', 'alpha2': 'EN'}, 'uniqueTournament': {'name': 'Premier League', 'slug': 'premier-league', 'category': {'name': 'England', 'slug': 'england', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'id': 1, 'country': {'alpha2': 'EN', 'alpha3': 'ENG', 'name': 'England', 'slug': 'england'}, 'flag': 'england', 'alpha2': 'EN'}, 'userCount': 1411277, 'id': 17, 'hasEventPlayerStatistics': True, 'displayInverseHomeAwayTeams': False}, 'priority': 616, 'id': 1}, 'season': {'name': 'Premier League 24/25', 'year': '24/25', 'editor': False, 'id': 61627}, 'roundInfo': {'round': 11}, 'customId': 'abskb', 'status': {'code': 100, 'description': 'Ended', 'type': 'finished'}, 'winnerCode': 1, 'homeTeam

In [ ]:
import pandas as pd
import time
import logging
from datetime import datetime
from tqdm import tqdm
import os
import json
from botasaurus.request import request, Request
from botasaurus_requests import response


DATE = '2024-11-09'  # You can change this as needed


# Initialize the Botasaurus session
@request(output=None, create_error_logs=False)
def _botasaurus_get(request: Request, url: str) -> response.Response:
    """SofaScore introduced some anti-scraping measures. Using Botasaurus gets around them."""
    if not isinstance(url, str):
        raise TypeError('`url` must be a string.')
    response = request.get(url)
    return response


def fetch_scheduled_events(date):
    url = f'https://www.sofascore.com/api/v1/sport/football/scheduled-events/{date}'
    response = _botasaurus_get(url)
    if response.status_code == 200:
        print(response.json())
        return response.json()
    else:
        print(f"Failed to fetch events for {date}")
        return None

scheduled_events = fetch_scheduled_events(DATE)


def filter_events_by_leagues(scheduled_events, leagues):
    """
    Filters events to only include those whose uniqueTournament ID is in the LEAGUES dictionary.

    Parameters:
    ----------
    scheduled_events : dict
        The dictionary containing all scheduled events.
    
    leagues : dict
        A dictionary of league names and their corresponding uniqueTournament IDs.
    
    Returns:
    -------
    list
        A filtered list of events.
    """
    all_events = scheduled_events.get('events', [])
    filtered_events = []

    for event in all_events:
        unique_tournament = event.get('tournament', {}).get('uniqueTournament', {})
        unique_tournament_id = unique_tournament.get('id')
        
        if unique_tournament_id in leagues.values():
            filtered_events.append(event)

    return filtered_events



def process_and_append_events(scheduled_events, csv_filename='europa_data_final.csv'):
    matches_data = []
    todays_games = []  # List to store today's games for comparison
    
    for event in tqdm(scheduled_events, desc="Processing events", unit="event"):
        try:
            event_id = event.get("id", None)
            home_team = event.get('homeTeam', {}).get('name', 'Unknown')
            home_team_id = event.get('homeTeam', {}).get('id', 'Unknown')
            away_team = event.get('awayTeam', {}).get('name', 'Unknown')
            away_team_id = event.get('awayTeam', {}).get('id', 'Unknown')
            home_score = event.get('homeScore', {}).get('display', 0)
            away_score = event.get('awayScore', {}).get('display', 0)

            start_timestamp = event.get("startTimestamp")
            if start_timestamp:
                match_datetime = datetime.fromtimestamp(start_timestamp)
                match_date = match_datetime.strftime("%Y-%m-%d")
                match_time = match_datetime.strftime("%H:%M:%S")
            else:
                match_date = None
                match_time = None

            gd = home_score - away_score
            if gd == 0:
                FTR = 1
            elif gd > 0:
                FTR = 0
            else:
                FTR = 2

            # Simulating match data scraping (replace this with the actual function call)
            match_data = ss.scrape_team_match_stats(event_id)
            match_data_filtered = match_data[['key', 'home', 'away']]

            new_row = {
                "eventId": event_id,
                "homeId": home_team_id,
                "homeTeam": home_team,
                "awayId": away_team_id,
                "awayTeam": away_team,
                "homeScore": home_score,
                "awayScore": away_score,
                "FTR": FTR,
                "date": match_date,
                "time": match_time
            }

            for index, row in match_data_filtered.iterrows():
                key = row['key']
                new_row[f'home{key}'] = row['home']
                new_row[f'away{key}'] = row['away']

            # Store today's games for comparison
            if match_date == DATE:
                todays_games.append(new_row)

            # Append data to CSV
            df = pd.DataFrame([new_row])
            if os.path.exists(csv_filename):
                df.to_csv(csv_filename, mode='a', index=False, header=False)
            else:
                df.to_csv(csv_filename, mode='w', index=False, header=True)

        except Exception as e:
            logging.error(f"Error processing event {event.get('id', 'Unknown')}: {e}", exc_info=True)
            continue  # Skip to the next event in case of an error

        time.sleep(1)

    # Create DataFrame of today's games
    todays_df = pd.DataFrame(todays_games)
    return todays_df


# Usage example
if scheduled_events:
    filtered_events = filter_events_by_leagues(scheduled_events, LEAGUES)
    todays_games_df = process_and_append_events(filtered_events)  # Collect today's games
    print("Today's games for comparison:")
else:
    print("No events available to process.")

    
# Fetch scheduled events
scheduled_events = fetch_scheduled_events(DATE)

if scheduled_events:
    filtered_events = filter_events_by_leagues(scheduled_events, LEAGUES)
    process_and_append_events(filtered_events)  # Pass the filtered events to your function
else:
    print("No events available to process.")

{'events': [{'tournament': {'name': 'Premier League', 'slug': 'premier-league', 'category': {'name': 'England', 'slug': 'england', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'id': 1, 'country': {'alpha2': 'EN', 'alpha3': 'ENG', 'name': 'England', 'slug': 'england'}, 'flag': 'england', 'alpha2': 'EN'}, 'uniqueTournament': {'name': 'Premier League', 'slug': 'premier-league', 'category': {'name': 'England', 'slug': 'england', 'sport': {'name': 'Football', 'slug': 'football', 'id': 1}, 'id': 1, 'country': {'alpha2': 'EN', 'alpha3': 'ENG', 'name': 'England', 'slug': 'england'}, 'flag': 'england', 'alpha2': 'EN'}, 'userCount': 1411277, 'id': 17, 'hasEventPlayerStatistics': True, 'displayInverseHomeAwayTeams': False}, 'priority': 616, 'id': 1}, 'season': {'name': 'Premier League 24/25', 'year': '24/25', 'editor': False, 'id': 61627}, 'roundInfo': {'round': 11}, 'customId': 'abskb', 'status': {'code': 100, 'description': 'Ended', 'type': 'finished'}, 'winnerCode': 1, 'homeTeam

Processing events:   0%|          | 0/60 [00:00<?, ?event/s]

Brentford
im here


Processing events:   2%|▏         | 1/60 [00:01<01:13,  1.25s/event]

Crystal Palace
im here


...

Sheffield United


Processing events:  98%|█████████▊| 59/60 [01:05<00:01,  1.06s/event]

Hull City


Processing events: 100%|██████████| 60/60 [01:06<00:00,  1.11s/event]

Processed 5846 events and appended them to europa_data_final.csv.
